# โหลดข้อมูล Testing

In [167]:
import pandas as pd
import numpy as np

Titanic_data = 'Titanic/test.csv' # เก็บชื่อที่อยู่ไฟล์
cols = ['Name', 'survived'] # คอลัมน์ที่ต้องการใช้
df = pd.read_csv(Titanic_data) # โหลดไฟล์
df.head(100) # preview

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
95,987,3,"Tenglin, Mr. Gunnar Isidor",male,25.0,0,0,350033,7.7958,NaN,S
96,988,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,19877,78.8500,C46,S
97,989,3,"Makinen, Mr. Kalle Edvard",male,29.0,0,0,STON/O 2. 3101268,7.9250,NaN,S
98,990,3,"Braf, Miss. Elin Ester Maria",female,20.0,0,0,347471,7.8542,NaN,S


In [168]:
df.shape # ดูขนาดข้อมูล

(418, 11)

In [169]:
X_test = df.Name # เลือกเฉพาะชื่อมาทำ features

# Data Preprocessing
- 1. ตัดคำแยกเป็น : LastName, Prefix. name
- 2. แยก column เป็น : prefix_name, LastName, Name
- 3. นับจำนวน : word_LastName, syllable_LastName, word_Name,syllable_Name
- 4. label encoding : prefix_name_, word_LastName_, syllable_LastName_, word_Name_, syllable_Name_
- 5. Normalizationให้ค่ามีscaleระหว่าง 0-1 : prefix_name_, word_LastName_, syllable_LastName_, word_Name_, syllable_Name_

# Data pipeline
- 1.Data Preprocessing
- 2.model_RFC predict X_test
- 3.Display

# Data Preprocessing(Code)

In [170]:
# ข้อมูล input
X = X_test

### 1.1.สำหรับเลือกชื่อมาใส่ list

In [173]:
# import library
from nltk.tokenize import WhitespaceTokenizer
import pandas as pd
import numpy as np

def preprocessing(X):
    
    ### 1. ตัดคำแยกเป็น : LastName, Prefix. name ###
    
    tokenized_lists = [] # สร้าง list ไว้รอเก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง
    for i in range(len(X)):
        tokenized_lists.append((WhitespaceTokenizer(), i)) # เก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง(ไว้ใช้ต่อไป)    
    results = []
    for tokenized, i in tokenized_lists:
        tokenized_list = tokenized.tokenize(X[i]) 
        results.append(tokenized_list) # เอาผลไว้ใน results
        
    ### 2. แยก column เป็น : LastName,Prefix_name. Name
    
    # เก็บข้อมูลลง DataFrame
    df_text = pd.DataFrame(results)  # ยังไม่ต้องระบุชื่อคอลัมน์
    # การตัดคำจะถูกแยกออกเป็นตัดแล้วได้ 1.ได้คำที่คอลัมน์ 1 พอดี 2. ไปอยู่ที่คอลัมน์ 2 แก้โดยหา idx_fail แล้วเปลี่ยน ณ index นั้นๆ
    # กำจัด missing data ด้วยการแทน str : ' '
    for i in range(len(df_text.columns)):
        df_text[i] = df_text[i].fillna(' ')
        
    Prefix_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.']
    Prefix_special = ''
    
    idx_fail = df_text[1].loc[
        
                          (df_text[1] != Prefix_list[0]) & (df_text[1]!= Prefix_list[1]) & (df_text[1] !=Prefix_list[2]) &  (df_text[1] !=Prefix_list[3]) &
                          (df_text[1] != Prefix_list[4]) & (df_text[1]!= Prefix_list[5]) & (df_text[1] !=Prefix_list[6]) &  (df_text[1] !=Prefix_list[7]) &
                          (df_text[1] != Prefix_list[8]) & (df_text[1]!= Prefix_list[9]) & (df_text[1] !=Prefix_list[10]) &  (df_text[1] !=Prefix_list[11]) &
                          (df_text[1] != Prefix_list[12]) & (df_text[1]!= Prefix_list[13]) & (df_text[1] !=Prefix_list[14]) & (df_text[1] !=Prefix_list[15])
        
                          ] 
            
    for i in idx_fail.index:
         for j in Prefix_list:
                # ตรวจดูข้อมูลแล้วเปลี่ยนให้ถูกต้องตามแต่กรณี
                if (df_text[2].loc[i] == j):
                    df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i] # รวมคอลัมน์ 0, 1
                    df_text[1].loc[i] = j                 # แทนที่คอลัมน์ 1 ด้วยคอลัมน์ 2
                    
                elif df_text[2].loc[i] == 'Countess.':
                    df_text[1].loc[i] = 'the Countess.' # แทนที่ the ด้วย 'the Countess.'
                    
                elif df_text[3].loc[i] == j:
                    df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i]+' '+ df_text[2].loc[i] # รวมคอลัมน์ 0, 1, 2
                    df_text[1].loc[i] = j          # แทนที่คอลัมน์ 2 ด้วยคอลัมน์ 3
                    
        
     
    
      

                                
      

    
      
              
        
            
            
        
        

        
 
    
    
    return df_text

In [174]:
df_text = preprocessing(X_test)

In [ ]:
len(X)

In [ ]:
df_text

In [ ]:
Prefix_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.']
len(Prefix_list)

In [188]:
df_text.sample(20) # ที่ index 414

,0,1,2,3,4,5,6,7
343,"Cardeza,",Mrs.,James,Warburton,Martinez,(Charlotte,Wardle,Drake)
413,"Spector,",Mr.,Woolf,,,,,
197,"Nilsson,",Miss.,Berta,Olivia,,,,
75,"Keeping,",Mr.,Edwin,,,,,
80,"Boulos,",Master.,Akar,,,,,
63,"Bradley,",Miss.,Bridget,Delia,,,,
46,"Chevre,",Mr.,Paul,Romaine,,,,
176,"Sincock,",Miss.,Maude,,,,,
154,"Asplund,",Master.,Filip,Oscar,,,,
235,"Vendel,",Mr.,Olof,Edvin,,,,


In [195]:
df_text[3].loc[414] 

'Dona.'

In [ ]:
df_text[2].unique() 